In [1]:
import os
import re
import pandas as pd
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

In [2]:
ETF = 'SPY'
ETF_3x = 'UPRO'

In [3]:
final_result = pd.read_excel(
    '{}_leverage_simulation.xlsx'.format(ETF),
    )

In [4]:
final_result.head(3)

,Date,SPY,simulated_UPRO,UPRO
0,1993-02-01,44.25000,0.501325,NaN
1,1993-02-02,44.34375,0.510006,NaN
2,1993-02-03,44.81250,0.553593,NaN


# the return

In [5]:
daily_invest_amount = 1000

invest = pysqldf(u"""
    select 
    Date,
    {} AS price,
    case
        when {} is not null then {}
        else simulated_{}
    end as price_3x
    from final_result
    """.format(
        ETF,
        ETF_3x,
        ETF_3x,
        ETF_3x,
    ))

invest1 = pysqldf(u"""
    select *,
    {} as daily_invest_amount,
    {}/price as shares,
    {}/price_3x as shares_3x    
    from invest
    order by Date asc
    """.format(
    daily_invest_amount,
    daily_invest_amount,
    daily_invest_amount))

In [6]:
invest1.head(3)

,Date,price,price_3x,daily_invest_amount,shares,shares_3x
0,1993-02-01,44.25000,0.501325,1000,22.598870,1994.713723
1,1993-02-02,44.34375,0.510006,1000,22.551092,1960.761835
2,1993-02-03,44.81250,0.553593,1000,22.315202,1806.380409


In [7]:
days_duration = int(12.5*250)

final_result['id'] = final_result.index

first_last_days = pysqldf(u"""
    select 
    f.Date as invest_date_first, 
    date_2500_days_later.Date as invest_date_last
    from final_result as f
    join final_result as date_2500_days_later on date_2500_days_later.id = f.id+{}
    """.format(days_duration)).to_dict('records')

print(days_duration)

3125


In [ ]:
returns = []

for r in first_last_days:
    invest_date_first = r['invest_date_first']
    invest_date_last = r['invest_date_last']
    
    print('processing data between {} and {}'.format(
    invest_date_first,
    invest_date_last,
    ))

    #invest_date_first = '2013-01-01'
    #invest_date_last = '2013-01-05'

    window = pysqldf(u"""
        select *
        from invest1
        where Date >= '{}'
        and Date <= '{}'
        """.format(
        invest_date_first, 
        invest_date_last
        ))

    total_df = pysqldf(u"""
        select 
        sum(daily_invest_amount) as invest_amount,
        sum(shares) as shares,
        sum(shares_3x) as shares_3x
        from window
        """)

    invest_amount = total_df['invest_amount'][0]
    shares = total_df['shares'][0]
    shares_3x = total_df['shares_3x'][0]

    ###

    first_date = pysqldf(u"""
        select *  
        from window
        order by Date asc
        limit 1
        """)

    first_price = first_date['price'][0]
    first_price_3x = first_date['price_3x'][0]

    last_date = pysqldf(u"""
        select *  
        from window
        order by Date desc
        limit 1
        """)

    last_price = last_date['price'][0]
    last_price_3x = last_date['price_3x'][0]

    ###

    all_in_return = last_price/first_price-1
    all_in_3x_return = last_price_3x/first_price_3x-1

    cost_averaging_return = shares*last_price/invest_amount-1
    cost_averaging_3x_return = shares_3x*last_price_3x/invest_amount-1

    record = {
        'invest_date_first':invest_date_first,
        'invest_date_last':invest_date_last,
        'all_in_return':all_in_return,
        'all_in_3x_return':all_in_3x_return,
        'cost_averaging_return':cost_averaging_return,
        'cost_averaging_3x_return':cost_averaging_3x_return,    
    }
    
    returns.append(record)


###    

returns_duration_df = pd.DataFrame(returns)

returns_duration_df.to_excel(
    '{}_returns_{}d.xlsx'.format(
        ETF,
        days_duration),
    index = False
    )

processing data between 1993-02-01 and 2005-06-27
processing data between 1993-02-02 and 2005-06-28
processing data between 1993-02-03 and 2005-06-29
processing data between 1993-02-04 and 2005-06-30
processing data between 1993-02-05 and 2005-07-01
processing data between 1993-02-08 and 2005-07-05
processing data between 1993-02-09 and 2005-07-06
processing data between 1993-02-10 and 2005-07-07
processing data between 1993-02-11 and 2005-07-08
processing data between 1993-02-12 and 2005-07-11
processing data between 1993-02-16 and 2005-07-12
processing data between 1993-02-17 and 2005-07-13
processing data between 1993-02-18 and 2005-07-14
processing data between 1993-02-19 and 2005-07-15
processing data between 1993-02-22 and 2005-07-18
processing data between 1993-02-23 and 2005-07-19
processing data between 1993-02-24 and 2005-07-20
processing data between 1993-02-25 and 2005-07-21
processing data between 1993-02-26 and 2005-07-22
processing data between 1993-03-01 and 2005-07-25


processing data between 1993-09-24 and 2006-02-21
processing data between 1993-09-27 and 2006-02-22
processing data between 1993-09-28 and 2006-02-23
processing data between 1993-09-29 and 2006-02-24
processing data between 1993-09-30 and 2006-02-27
processing data between 1993-10-01 and 2006-02-28
processing data between 1993-10-04 and 2006-03-01
processing data between 1993-10-05 and 2006-03-02
processing data between 1993-10-06 and 2006-03-03
processing data between 1993-10-07 and 2006-03-06
processing data between 1993-10-08 and 2006-03-07
processing data between 1993-10-11 and 2006-03-08
processing data between 1993-10-12 and 2006-03-09
processing data between 1993-10-13 and 2006-03-10
processing data between 1993-10-14 and 2006-03-13
processing data between 1993-10-15 and 2006-03-14
processing data between 1993-10-18 and 2006-03-15
processing data between 1993-10-19 and 2006-03-16
processing data between 1993-10-20 and 2006-03-17
processing data between 1993-10-21 and 2006-03-20


def calculate_return(
    invest_date_first,
    invest_date_last,
    ):

pysqldf(u"""
    select *  
    from invest
    where Date in ('2001-04-24', '2009-12-15')
    """)

return_df.plot(
    x ='Date', 
    y=[
        'accumulated_return', 
        'accumulated_return_3x', 
    ], grid = True)

return_df.plot(
    x ='Date', 
    y=[
        'price', 
        'price_3x', 
    ], grid = True)

END